# Importing basic libraries

In [1]:
import numpy as np                          #function is now called np
import pandas as pd                         #function is now called pd 
import requests                             #makes http requests
import xlsxwriter                           #creates excel report
import math                                 #executes operations

# Importing List of Stock

In real life you will use API to get price data.
Here we use a sample CSV file of S&P500 stocks

S&P 500 data is in 'starter-files' folder 

In [2]:
stocks = pd.read_csv(r"C:\Users\nik10\iCloudDrive\Documents\Python Trading Robot\Project 1 - Equal Weight Index Fund\starter_files\sp_500_stocks.csv")

# Acquiring an API Token

API token is the data provider to be used - token should be stored in secrets.py

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

# Making Our First API CALL

Info needed 
* Market cap of each stock
* Price of each stock

IEX Cloud api is used to get data - https://iexcloud.io/docs/api/ 

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' 
#base url is used to initiate https request - found in API Reference

data = requests.get(api_url).json()
#use request library for http requests, use json to get accessible data format for 'data' - in this case dictionary
#       print(data)

# Parsing Our API CAll
Previous API call contains all info needed for S&P 500 Strategy

Parsing - convert data into a proper format

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

# Adding Stock Market Data to a Pandas DataFrame
We want to create a spreadsheet that will store the stock price and market cap

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy'] 
# Declaring the columns for data frame

final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,151.8,2514494114449,N/A


# Creating a loop that goes through a list of stocks

We pull data for all S&P 500 stocks and store the data in the DataFrame using `for` loop

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:10]:
    #create api call for each cell - very slow as makes a http request for each stock (to be optimised)
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True    
    )

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,169.41,51372739891,N/A
1,AAL,18.83,12518024215,N/A
2,AAP,212.52,14258636870,N/A
3,AAPL,147.40,2521778349350,N/A
4,ABBV,121.35,212868446258,N/A
5,ABC,120.50,24992065928,N/A
6,ABMD,334.80,15254127900,N/A
7,ABT,128.09,233309084771,N/A
8,ACN,332.97,225555267284,N/A
9,ADBE,642.05,318121938556,N/A


# Using Batch API Calls to Improve Perfomance

API providers will give discounts for using batch API
IEX cloud has 100 tickets limit per request - we split stocks into groups of 100 to make a call

We want to split a list into sublists

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

Arrange tickers into 5 groups of 5 strings

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #use split method to undo join
    for symbol in symbol_string.split(','):
#        print (symbol)
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True    
    )
final_dataframe
#1:09:56

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,172.85,52130699370,N/A
1,AAL,19.48,12452343382,N/A
2,AAP,217.26,13754436422,N/A
3,AAPL,148.50,2430944632310,N/A
4,ABBV,120.74,209457922920,N/A
...,...,...,...,...
500,YUM,140.56,40645542581,N/A
501,ZBH,146.17,31399222779,N/A
502,ZBRA,567.64,30723645350,N/A
503,ZION,54.98,8700645882,N/A


# Calculating the number of shares to buy
This part creates a field where we enter the size of our portfolio - easy to adapt for other portfolios

In [12]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    #forces portfolio_size variable to become float
#    print (val)
except ValueError:
    print('Only numerical characters are allowed! \nPlease try again:')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000


In [13]:
#portfolio_size
#output is a string
position_size = val/len(final_dataframe.index)
#   print(position_size)
#shows how much money should be invested in each stock

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe
#    print(math.floor(number_of_apple_shares)) #rounds down each position size


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,172.85,52130699370,0
1,AAL,19.48,12452343382,1
2,AAP,217.26,13754436422,0
3,AAPL,148.50,2430944632310,0
4,ABBV,120.74,209457922920,0
...,...,...,...,...
500,YUM,140.56,40645542581,0
501,ZBH,146.17,31399222779,0
502,ZBRA,567.64,30723645350,0
503,ZION,54.98,8700645882,0


# Creating format will export `.xlsx` file
XlsxWriter library will create Excel file

In [14]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
# Pandas recognise Excel Writer

final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

# Creating the format We'll Need For Our `.xlsx` File
Formats include colours, fonts, symbols. Main formats needed:

* String format for tickers
* \$ XX.XX format for stock prices
* \$ XX.XXX format for market capitalization
* Integer format for the number of shares to buy

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

# Applying the Formats to the Columns of Our `.xlsx` File

Use `set_column` method applied to the `writer.book` object to apply formats to specific columns of our spreadsheets.

In [16]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()
# #Ticker column

# writer.sheets['Recommended Trades'].write('A1','Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1','Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1','Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy', integer_format)


Simplify by using loops

In [17]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0], column_formats[column][1])

# Saving our Excel file


In [18]:
writer.save()  
#1:38:43